In [65]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy

In [66]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [67]:
nlp = spacy.load('en_core_web_sm')

In [68]:
from spacy.lang.en.stop_words import STOP_WORDS as stopwords

In [69]:
print(STOP_WORDS)

{'whom', 'be', 'his', '’d', 'four', 'of', 'somewhere', 'my', 'n’t', 'into', 'few', 'same', 'seeming', 'serious', 'doing', 'being', 'together', 'formerly', 'become', 'would', 'much', 'when', 'without', 'are', 'regarding', 'besides', 'yet', 'least', 'nor', 'up', 'nine', '‘re', 'therefore', 'otherwise', 'were', 'which', 'third', 'along', 'whether', 'made', 'alone', 'put', 'wherein', 'somehow', 'any', 'five', 'her', 'either', '‘s', 'until', 'others', 'another', 'anyone', 'done', 'them', 'hers', 'before', 'where', 'also', 'still', 'hundred', 'toward', 'yours', 'anywhere', 'below', 'whereafter', 'some', 'across', 'name', 'already', 'cannot', 'onto', 'forty', 'more', '’re', 'behind', 'does', 'namely', 'beforehand', 'have', 'for', 'its', 'all', 'over', 'amount', 'while', 'between', 'throughout', 'hereafter', 'off', 'back', 'it', 'themselves', 'however', 'could', 'each', 'just', 'most', 'almost', 'former', 'became', 'because', 'else', 'call', 'via', 'towards', 'eleven', 'from', 'they', 'within'

In [70]:
len(STOP_WORDS)

326

In [71]:
data_yelp = pd.read_csv('Dataset/yelp_labelled.txt', sep='\t', header=None)

In [72]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [73]:
column_names = ['Review', 'Sentiment']
data_yelp.columns = column_names

In [74]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [75]:
data_yelp.shape

(1000, 2)

In [76]:
data_amazon = pd.read_csv('Dataset/amazon_cells_labelled.txt', sep='\t', header=None)
data_amazon.columns = column_names

In [77]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [78]:
data_amazon.shape

(1000, 2)

In [79]:
data_imdb = pd.read_csv('Dataset/imdb_labelled.txt', sep='\t', header=None)
data_imdb.columns = column_names

In [80]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [81]:
data_imdb.shape

(748, 2)

In [82]:
data = data_yelp.append([data_amazon, data_imdb], ignore_index=True)

In [83]:
data.shape

(2748, 2)

In [84]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [85]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [86]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

# Tokenization

In [87]:
import string

In [88]:
punct = string.punctuation

In [89]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [90]:
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [91]:
text_data_cleaning("Hello how are you. This is amazing video on NLP")

['hello', 'amazing', 'video', 'nlp']

# TF-IDF

In [92]:
from sklearn.svm import LinearSVC

In [93]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)

In [94]:
classifier = LinearSVC()

In [95]:
X = data['Review']
y = data['Sentiment']

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [97]:
X_train.shape , X_test.shape

((1923,), (825,))

In [98]:
X_train

1560    Muddy, low quality sound, and the casing aroun...
1777                                          WORTHWHILE.
1228    I have had this phone for over a year now, and...
307     As much as I'd like to go back, I can't get pa...
812     This place is overpriced, not consistent with ...
                              ...                        
1638    I used bitpim (a free program you can find on ...
1095                          Will order from them again!
1130    It is so small and you don't even realize that...
1294    The voice recognition thru the handset is exce...
860     This place is pretty good, nice little vibe in...
Name: Review, Length: 1923, dtype: object

In [99]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])

In [100]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x0000021144515678>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_inter

In [101]:
y_pred = clf.predict(X_test)

In [102]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.80      0.79       435
           1       0.77      0.75      0.76       390

    accuracy                           0.78       825
   macro avg       0.78      0.78      0.78       825
weighted avg       0.78      0.78      0.78       825



In [103]:
confusion_matrix(y_test, y_pred)

array([[349,  86],
       [ 98, 292]], dtype=int64)

In [104]:
clf.predict(['This was amazing movie.'])

array([1], dtype=int64)

In [106]:
clf.predict(['This movie boring.'])

array([0], dtype=int64)